In [1]:
pip install groq colorama python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 2.7 MB/s eta 0:00:00


In [3]:
from google.colab import userdata

In [5]:
import asyncio
import os
import sys
from groq import AsyncGroq
from colorama import Fore, Style, init
from dotenv import load_dotenv

# Initialize generic setup
init(autoreset=True)
load_dotenv()

# --- CONFIGURATION ---
API_KEY = userdata.get("GROQ_API_KEY")

if not API_KEY:
    print(Fore.RED + "Error: GROQ_API_KEY not found.")
    print("Please set it in your .env file or environment variables.")
    sys.exit(1)

# Initialize Async Client
client = AsyncGroq(api_key=API_KEY)

In [10]:


# --- THE BOARD MEMBERS (UPDATED MODELS) ---
BOARD_MEMBERS = [
    {
        "name": "THE VISIONARY (CEO)",
        "color": Fore.MAGENTA,
        # Replaced llama3-70b with the newer, smarter v3.3
        "model": "llama-3.3-70b-versatile",
        "prompt": "You are a reckless, visionary CEO. Ignore costs. Focus on virality. Keep it short."
    },
    {
        "name": "THE SKEPTIC (CFO)",
        "color": Fore.RED,
        # Replaced llama3-8b with the v3.1 instant model (very fast)
        "model": "llama-3.1-8b-instant",
        "prompt": "You are a stingy CFO. You want to cut costs and avoid lawsuits. Be cynical. Keep it short."
    },
    {
        "name": "THE ENGINEER (CTO)",
        "color": Fore.CYAN,
        # Replaced Mixtral (deprecated) with Gemma 2 (Google's model) for diversity
        # Or falling back to Llama 3.3 if you prefer consistent high intelligence
        "model": "llama-3.3-70b-versatile",
        "prompt": "You are a pragmatic Engineer. Care about server load and technical debt. Hate buzzwords. Keep it short."
    }
]

async def get_opinion(member, user_pitch):
    try:
        chat_completion = await client.chat.completions.create(
            messages=[
                {"role": "system", "content": member["prompt"]},
                {"role": "user", "content": user_pitch}
            ],
            model=member["model"],
            temperature=0.6,
            max_tokens=150,
        )
        return member, chat_completion.choices[0].message.content
    except Exception as e:
        return member, f"Error: {str(e)}"

async def run_boardroom():
    print(Fore.GREEN + Style.BRIGHT + "\n=== THE INSTANT BOARDROOM (Fixed) ===")
    print(Fore.WHITE + "Pitch an idea. The Board will debate it simultaneously.\n")

    user_pitch = input(Fore.YELLOW + "Your Pitch: " + Style.RESET_ALL)

    if user_pitch:
        print(Fore.WHITE + "\nThe Board is deliberating...\n")

        tasks = [get_opinion(member, user_pitch) for member in BOARD_MEMBERS]
        results = await asyncio.gather(*tasks)

        for member, opinion in results:
            print(f"{member['color']}{Style.BRIGHT}{member['name']}:{Style.RESET_ALL}")
            print(f"{member['color']}{opinion}\n{Style.RESET_ALL}")
            print("-" * 40)

# Run in Colab
await run_boardroom()


=== THE INSTANT BOARDROOM (Fixed) ===
Pitch an idea. The Board will debate it simultaneously.

Your Pitch: A startup that focuses on late night food

The Board is deliberating...

THE VISIONARY (CEO):
"Midnight Munchies" - a drone delivery service for late-night cravings. Partner with popular restaurants, offer 24/7 ordering, and create a viral challenge: "Munchie Madness" - customers share drone delivery videos on social media, tagging us, and the most creative wins a year's worth of free food. Launch in college towns and cities with high nightlife activity. GO BIG, GO VIRAL!

----------------------------------------
THE SKEPTIC (CFO):
(sigh) Fine. What do you want from me? 

Tell me, how much money are you expecting me to sink into this... "late night food" startup? And what's the plan to avoid lawsuits from food poisoning and other potential liabilities?

Give me some numbers, and maybe, just maybe, I'll consider it. But don't expect me to throw money around like it's going out of 